# Assignment 1 – Video to Event Stream (35%)

## Q1. (75%) Build an algorithm to convert a video sequence to an event stream with a reasonable SNR and high temporal resolution.

To start this script I use numpy, openCV and event_stream as they provide all the neccessary bindings to complete most of the computation outside of native Python.

In [ ]:
from numpy.lib.recfunctions import unstructured_to_structured
import cv2 as cv
import numpy as np
import event_stream

Like with most algorithms the code starts with initializing parameters before entering a loop. The code begins by getting the relative path for the desired video. OpenCV is used to extract the data from the MP4 file. Initial parameters for FPS, frame period are retrieved from the data. frameTime is initialized and works as an incrementor with frameInterval steps. The first frame of the recording is stored before the loop as a reference for the first difference calculation.

In [ ]:
video_src = 'Pexels Videos 2629.mp4' # Video Relative Source Path
data = cv.VideoCapture(video_src) # open cv read mp4 file

fps = data.get(cv.CAP_PROP_FPS) # Get Video Frame Rate
frameInterval = 1/fps # Get Frame Period

frameTime = 0 # Initial Frame Time Incrementer

ret, img = data.read()
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY).astype('int16')
prevImg = img

Threshold for event spikes is set in this variable.

In [ ]:
thresh = 30

For this assignment I decided to convert the MP4 videos to ES files as the performance was evaluated to be much better than converting to CSV which is explained and evaluated in the report. The numpy array prototype is created in dtype and the event_stream encoder is initialized with information from the first frame of the recording.

In [ ]:
dtype = np.dtype([('t', '<u8'), ("x", '<u2'), ("y", '<u2'), ("on", '?')])
#with open(f"EventOutput\\{video_src[:-4]}.csv", 'a') as csvFile:
        #np.savetxt(csvFile, np.zeros(1, dtype=dtype), delimiter=',', fmt=["%d","%d","%d","%d"], header="timestamp,x,y,polarity") # Write header out to file
resX = np.shape(img)[0]
resY = np.shape(img)[1]
encoder = event_stream.Encoder(f"EventOutput\\{video_src[:-4]}.es", 'dvs', resX, resY)

## Algorithm

The main loop of this algorithm is split into two components. The first component is direct frame analysis. In direct frame analysis the direct frame to frame difference is computed and events are extracted directly from the differences. Direct frame analysis provides event-based data however it provides incomplete data as events that likely happened between frames are rejected. The second component addresses this by interpolating events between frames. This component is the most computationally expensive aspect of this algorithm.

### Direct Frame Analysis

The algorithm begins by loading the current frame data and the previous frame data as numpy arrays of gray-scale values between 0 and 255.

The arrays are subtracted to find the gray-scale difference between frames.

A mask of all differences larger than the specified threshold is created and the difference array is masked.

The value and x and y index of all nonzero values in the masked difference array is collected.

At this stage the data becomes event-based as the algorithm no longer deals with the complete spatial array of the frame and purely the values with recorded differences.

An important characteristic of the current data is that the value encodes both the number of spikes expected and the polarity of the spikes. Positive values encode ON events and negative values encode OFF events.

To get the interpolated spike counts, the data is divided by the threshold value and rounded to the closest 16 bit integer. Rounding this way ensures that the lowest absolute integer is chosen for each value (e.g. 5.5 >> 5 and -5.5 >> -5). 

To get polarity information out of the data, The real value is divided by the absolute value. This will return 1 or -1. The output is compared to 1 which will return 1 or 0 which is the format that polarity data should be in.

The last data extraction done in Direct Frame Analysis is to get the timestamp for the current frame. The iterator frameTime is converted to microseconds and copied to match the same length of the polarity and x,y arrays.

Finally the arrays are all joined together and formatted into a structured array to be fed into the event_stream encoder.


### Inter-frame Event Interpolation

An important aspect of event data is its asynchronous temporal distribution. To emulate this, interpolation shouldn't be computed using a global event interval between frames. A good way to avoid this is to dynamically create event intervals depending on the input data.

The first step to get start the event interval generation is to find the maximum spike count found from Direct Frame Analysis.

The spike time delta is computed by dividing the frame interval by the spike count from Direct Frame Analysis.

The interpolated event chunk is initialized.

This is where the dynamic aspect of the algorithm works. The data is looped through for each event interval that can exist between 2 events and the maximum number of events recorded.

In the loop, temporary arrays for timestamp, coordinates and polarity are used and at the end of each loop are appended to the event chunk.

The timestamp relative to the current timestamp is calculated by multiplying the spike time delta by the current loop iteration.

Each temporary array in the loop is indexed by comparing the interpolated spike count array against the current loop count. If the interpolated spike count is larger than the loop count, the data is kept and if the loop count is larger, the data is disregarded. A product of this process is that the temporary arrays describe less data as the loop iterates. Looping through the data this way ensures that each interpolated spike is distributed evenly in time and is independent of it's neighbors or a global event interval.

After the loop is completed the timestamps are converted to global timestamps and the data in the event chunk is structured to match the requirements for the event_stream encoder.

The unfortunate by-product of this method is that the event chunk is only partially sorted by timestamp.

The event chunk is sorted using Timsort as it is optimal for data that is already partially sorted.

The final stage of the algorithm is to join the events found from direct frame analysis with the interpolated events.

The structured array is then encoded into the Event Stream File and the frame time is iterated and the current image data is stored as reference for the next loop.




In [ ]:
while(data.isOpened()):
    ret, img = data.read()

    if type(img) == type(None):
        break

    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY).astype('int16')

    eventMask = cv.compare(cv.absdiff(img, prevImg), thresh, cv.CMP_GT)

    change = cv.subtract(img, prevImg) # Get light difference

    eventFrame = cv.bitwise_and(change, change, mask = eventMask) # Get Event Data in frame
    
    eventCoords = np.nonzero(eventFrame) # Get coordinates of all nonzeros in frame mask

    eventData = eventFrame[eventFrame != 0] #Get values of all nonzeros in frame mask

    eventSpikes = np.asarray(np.divide(eventData, thresh), dtype=np.int16) # Interpolate number of spikes between frames

    spikeNum = np.abs(eventSpikes) # get total number of interpolated events for each pixel between frames
    
    polarity = np.equal(np.divide(eventSpikes, spikeNum), 1) # get polarity of events for each pixel
    
    currentTmstp = np.full(np.shape(polarity), frameTime*1000000) # make array of timestamps

    Events = np.stack((currentTmstp, eventCoords[0], eventCoords[1], polarity)) # join all arrays together
    Events = np.transpose(Events) #Transpose array
    Events = unstructured_to_structured(Events, dtype=dtype) # Fit array to structured array

    ##########################Interpolation#####################################
    if frameTime != 0:
        spikeMax = np.max(spikeNum) # Get maximum number of interpolated spikes found between frames
        spikeTimeDelta = np.divide(frameInterval, spikeNum) # get spike time delta for each interpolated event between frames

        eventChunk = [] #Initialise Interpolation List

        for i in np.arange(1,spikeMax): # Loop up to maximum spike times found

            timeTemp = np.multiply(spikeTimeDelta, i*1000000) #Create array of interpolated time deltas
            timeTemp = np.asarray(timeTemp[spikeNum > i],dtype=np.float32) # remove interpolated time deltas that don't correspond to events

            polarityTemp = polarity[spikeNum > i] # Create array of polarities of interpolated events
            
            coordXTemp = np.asarray(eventCoords[0][spikeNum > i], dtype=np.uint16) # Create array of interpolated event X coordinates
            coordYTemp = np.asarray(eventCoords[1][spikeNum > i], dtype=np.uint16) # Create array of interpolated event Y coordinates

            event = np.stack((timeTemp, coordXTemp, coordYTemp, polarityTemp)) # Join interpolated information into single array
            eventChunk.append(event) # add chunk of interpolated events to list

        interpolatedEvents = np.concatenate(eventChunk, axis=1) # Concatenate all event chunks from list
        intpTmstp = np.full((1, np.size(interpolatedEvents, 1)), frameTime*1000000) # Create timestamp array the same size as the interpolated events
        interpolatedEvents[0,:] = np.subtract(intpTmstp, interpolatedEvents[0,:]) # Calculate time delta from current timestamp of interpolated events

        interpolatedEvents = np.transpose(interpolatedEvents) # Transpose interpolated event array
        interpolatedEvents = unstructured_to_structured(interpolatedEvents, dtype=dtype) # Fit array to structured array

        interpolatedEvents = np.sort(interpolatedEvents ,order='t', kind='stable') # Sort Structured Interpolated Array by timestamp

        Events = np.concatenate((interpolatedEvents, Events)) # Join Interpolated Events and Regular Events

    encoder.write(Events) # Write array out to ES file

    #with open(f"EventOutput\\{video_src[:-4]}.csv", 'a') as csvFile:
        #np.savetxt(csvFile, Events, delimiter=',', fmt=["%d","%d","%d","%d"]) # Write array out to file
        
  
    frameTime += frameInterval
    prevImg = img

    

data.release()

## Q2. (15%) Evaluate and briefly justify your algorithms

### Evaluation